# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846609818434                   -0.70    4.8         
  2   -7.852291466038       -2.25       -1.53    1.0   23.8ms
  3   -7.852607780231       -3.50       -2.56    1.5   25.6ms
  4   -7.852645822343       -4.42       -2.88    2.5   32.6ms
  5   -7.852646483376       -6.18       -3.16    1.0   86.2ms
  6   -7.852646677571       -6.71       -4.09    1.0   25.7ms
  7   -7.852646686516       -8.05       -4.81    2.2   31.2ms
  8   -7.852646686718       -9.69       -5.34    1.8   27.4ms
  9   -7.852646686727      -11.06       -5.51    1.5   26.4ms
 10   -7.852646686730      -11.51       -6.44    1.0   24.2ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846918479895                   -0.70           4.8         
  2   -7.852527712052       -2.25       -1.64   0.80    2.0    287ms
  3   -7.852636030650       -3.97       -2.73   0.80    1.0   22.5ms
  4   -7.852646460459       -4.98       -3.24   0.80    2.2   29.5ms
  5   -7.852646667029       -6.68       -4.05   0.80    1.0   21.9ms
  6   -7.852646686313       -7.71       -4.70   0.80    1.8   25.8ms
  7   -7.852646686723       -9.39       -5.41   0.80    1.8   26.7ms
  8   -7.852646686729      -11.24       -6.69   0.80    1.2   22.8ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.345882e+01     3.278117e+00
 * time: 0.06934499740600586
     1     1.523815e+00     1.732013e+00
 * time: 0.30182909965515137
     2    -1.358527e+00     1.968012e+00
 * time: 0.32564711570739746
     3    -3.633271e+00     1.846355e+00
 * time: 0.3600740432739258
     4    -5.004065e+00     1.557205e+00
 * time: 0.47760605812072754
     5    -6.711159e+00     1.208558e+00
 * time: 0.5120320320129395
     6    -7.408951e+00     9.299443e-01
 * time: 0.5458269119262695
     7    -7.687954e+00     8.265741e-01
 * time: 0.5692570209503174
     8    -7.772886e+00     5.453095e-01
 * time: 0.5929510593414307
     9    -7.813269e+00     2.700745e-01
 * time: 0.6174769401550293
    10    -7.830297e+00     9.166179e-02
 * time: 0.6415960788726807
    11    -7.843210e+00     9.554148e-02
 * time: 0.6653211116790771
    12    -7.847756e+00     6.376009e-02
 * time: 0.6891219615936279
    13    -7.850397e+00     2.895699e-02
 * time: 0.712492

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846812399343                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645824595                   -1.64         
  2   -7.852646686730       -6.06       -3.70    2.11s
  3   -7.852646686730      -13.11       -7.18    172ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.420492148073578e-7
|ρ_newton - ρ_scfv| = 4.824984516274362e-7
|ρ_newton - ρ_dm|   = 8.646809892449614e-10
